# Install And Import dependency

In [1]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 832.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 36.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.13.1+cu116
    Uninstalling torchaudio-0.13.1+cu116:
      Successfully uninstalled torchaudio-0.13.1+cu116
ERROR: pip's dependency resolver does not

In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.7 MB/s eta 0:00:00


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re



1.   Beautiful Soup is going to help to extact data from the reviews
2.   



# Instantiate Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


# Encode and Calculate Sentiment

In [5]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [11]:
tokens[0] # tokenizer is parsing the text into sequence of number 

tensor([  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
        11838,   102])

In [14]:
# to re convert the token to sentence
tokenizer.decode(tokens[0])

'[CLS] it was good but couldve been better. great [SEP]'

In [16]:
result = model(tokens) # passsing the tokens to the model 
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)


the values in the 'logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]] ' represents the probabilty of a particular class being the sentiment.

In [7]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [18]:
int(torch.argmax(result.logits))+1 # torch.argmax returns the highest value.
# we have value ranging between 1,2,3,4 
# the higher the number, the better it is. 

4

# Collect Reviews

In [19]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [24]:
results[0].text

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [23]:
soup.find_all('p', {'class':regex}) # wrapped in a p tag has a comment. 

[<p class="comment__09f24__gu0rG css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly</span></p>,
 <p class="comment__09f24__gu0rG css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Great staff and food.  Must try is the pan fried Gnocchi!  The staff were really friendly and the coffee was good as well</span></p>,
 <p class="comment__09f24__gu0rG css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Ricotta hot cakes! These were so yummy. I ate them pretty fast and didn't share with anyone because they were that good ;). <br/><br/>I ordered a green smoothie to balance it all out. Smoothie was a nice way to end my brekkie at this restaurant. <br/><br/>Others with me ordered the salmon Benedict and the smoked salmon flatbread. They were all delicious and all plates were empty. <br/><br/>Cheers!</span></p>,
 <p class="comment__09f24__gu0rG css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">I came

In [21]:
reviews[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

# Load Reviews into DataFrame and Score

In [25]:
import numpy as np
import pandas as pd

In [26]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [34]:
df.head(3) # stroing the reviews in the dataframe

,review,sentiment
0,Great food amazing coffee and tea. Short walk ...,5
1,Great staff and food. Must try is the pan fri...,5
2,Ricotta hot cakes! These were so yummy. I ate ...,5


In [35]:
df['review'].iloc[0]

'Great food amazing coffee and tea. Short walk from the harbor. Staff was very friendly'

In [36]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [31]:
sentiment_score(df['review'].iloc[1]) # sending the review to the dataframe

5

In [32]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))
